# FASAM - CHURN COMPETITION 
Alunos: Wagner Cristian de Siqueira / Jackson Dias Savitraz 
 
## Introdução 
O *churn*, também conhecido como *churn rate*, é um indicador que mede o índice de evasão dos clientes, ou seja, a taxa de cancelamento. 
Isso quer dizer que o famoso *churn* é o responsável por dizer para os gestores quantos clientes eles estão perdendo, seja por mês, a cada 6 meses ou por ano. 
Ele é importante porque, se sua empresa está perdendo muitos clientes, ou seja, está com o *churn* alto, é sinal de que há algum problema que precisa ser melhorado. 
Pode ser o seu produto, que não está adequado às expectativas dos clientes, pode ser o atendimento da sua equipe, que está deixando a desejar… enfim, é preciso investigar onde está o ponto fraco. 
 
## Objetivo 
 Um banco está investigando uma taxa muito alta de churn dos seus clientes. 
 Aqui está um conjunto de dados de 10.000 registros para investigar e prever quais clientes estão mais propensos a sair do banco em breve. 
  
## Modelo
O XGBoost (*Extreme Gradient Boosting*), baseado no projeto de pesquisa "*XGBoost: A Scalable Tree Boosting System*" proposto por Tianqi Chen e Carlos Guestrin, é o algoritmo mais popular de machine learning atualmente. 
Utiliza um conjunto de métodos baseados em árvores de decisão, reunidos em uma biblioteca projetada e otimizada para para extrair o máximo de performance das arquiteturas computacionais disponíveis e criar um modelo mais geral. 
O mecanismo utilizado pelo XGBoost como modelo de escolha é chamado *ensemble* de árvores de decisão, ou seja, um conjunto de árvores de classificação e regressão denominado CART (*Classification and Regression Trees*). 
Com integração para Python através do framework Scikit-learn, este algoritmo é o estado da arte para problemas de classificação e ganhou muita popularidade devido ser escolha de muitas equipes vencedoras em competições de aprendizagem de máquina no Kaggle. 
 
## Conclusão
Através da utilização do modelo que atualmente é o estado da arte em problemas de classificação, conseguimos um F-Score médio maior que 0,9 para o dataset de validação e, consequentemente, o primeiro lugar do desafio.

In [0]:
#importa as bibliotecas
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, f1_score
from sklearn.model_selection import train_test_split
from xgboost.sklearn import XGBClassifier

#função para verificar o treinamento do modelo
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred.round()))
    mae  = mean_absolute_error(actual, pred.round())
    r2   = r2_score(actual, pred.round())
    f1   = f1_score(actual, pred.round())
    print("RMSE:", rmse)
    print("MAE:", mae)
    print("R2:", r2)
    print("F1:", f1)

#função para salvar o arquivo de submissão
def save_submission(df, name='submission.csv'):
    with open(name, 'w+') as f:
        f.write("RowNumber,Exited\n")    
        i = 0
        for k, row in df.iterrows():
            f.write("{},{}\n".format(row.RowNumber, row.Exited))  
            i = i+1

In [0]:
#carrega o dataset de treino
dataset = pd.read_csv('train.csv')

#para as entradas, descartamos as colunas RowNumber, CustomerId, Surname e Exited
X = dataset.iloc[:,3:-1].values

#para a saída, utilizamos a coluna Exited
y = dataset.iloc[:,-1].values

#utiliza o label encoder para as colunas Geography e Gender
le = LabelEncoder()
X[:,1] = le.fit_transform(X[:,1])
X[:,2] = le.fit_transform(X[:,2])

#utiliza o one-hot encoder para gerar as colunas de features
preprocessor = make_column_transformer((OneHotEncoder(categories='auto'),[1]),remainder='passthrough')
X = preprocessor.fit_transform(X)

#utiliza o standard scaler para normalizar as colunas valoradas
sc = StandardScaler()
X = sc.fit_transform(X)

In [0]:
#quebra o dataset em treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [47]:
#treina o modelo com as entradas e saída
model = XGBClassifier(max_depth=8)
model.fit(X_train, y_train)

#verifica o treinamento do modelo
y_pred = model.predict(X_test)
eval_metrics(y_test, y_pred)

RMSE: 0.36639108100829126
MAE: 0.13424242424242425
R2: 0.14431259661529683
F1: 0.5840375586854459


In [0]:
#carrega o dataset de validação
dataset = pd.read_csv('valid.csv')

#para as entradas, descartamos as colunas RowNumber, CustomerId e Surname
X_valid = dataset.iloc[:,3:].values

#utiliza o label encoder para as colunas Geography e Gender
X_valid[:,1] = le.fit_transform(X_valid[:,1])
X_valid[:,2] = le.fit_transform(X_valid[:,2])

#utiliza o one-hot encoder para gerar as colunas de features
X_valid = preprocessor.fit_transform(X_valid)

#utiliza o standard scaler para normalizar as colunas valoradas
X_valid = sc.fit_transform(X_valid)

#treina o modelo com todo dataset
model.fit(X, y)

#faz a predição do dataset de validação
pred = model.predict(X_valid)

#armazena o valor gerado para a coluna Exited
dataset['Exited'] = pred

#salva o arquivo de submissão para o Kaggle
save_submission(dataset)